In [44]:
import sys
import csv
from selenium import webdriver
import time

# default path to file to store data
path_to_file = "/Users/didie/Documents/Research/IGNITION - Hotels Review - Seb/Reviews_Restaurants_85.csv"

# default number of scraped pages
num_page = 1000

# default tripadvisor website of restaurant
url = "https://www.tripadvisor.com/Restaurant_Review-g55711-d649839-Reviews-Pappas_Bros_Steakhouse-Dallas_Texas.html"
# if you pass the inputs in the command line
if (len(sys.argv) == 4):
    path_to_file = sys.argv[1]
    num_page = int(sys.argv[2])
    url = sys.argv[3]

PATH = "/usr/bin/chromedriver"
# Import the webdriver
driver = webdriver.Chrome(PATH)
driver.get(url)

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Open the file to save the review
# csvFile = open(path_to_file, 'a', encoding="utf-8")
# csvWriter = csv.writer(csvFile)

def expand_reviews():
    element = WebDriverWait(driver, timeout=10).until(
            EC.presence_of_element_located((By.XPATH, "//span[@class='taLnk ulBlueLinks']")) 
    )
    if element is not None:
        element.click() 
    

# change the value inside the range to save more or less reviews
for i in range(0, 1000):
    
#     # expand the review 
    try:
        expand_reviews()
    except Exception as e:
        expand_reviews()

#     driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
    time.sleep(2)
    
    container = driver.find_elements_by_xpath(".//div[@class='review-container']")
    
    print(f"Num reviews in this page: {len(container)}")
#     print(container)
    for j in range(len(container)):
#         print(container[j].text)
        title = container[j].find_element_by_xpath(".//span[@class='noQuotes']").text
        date = container[j].find_element_by_xpath(".//span[contains(@class, 'ratingDate')]").get_attribute("title")
        rating = container[j].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        try:
            review = container[j].find_element_by_xpath(".//span[@class='postSnippet']").text.replace("\n", " ")
        except:
            review = container[j].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", " ")
        
        print([date, rating, title, review]) 
        res = input("continue? [y/n]: ")[0]
        if res == "n":
            break
            
    # change the page
    driver.find_element_by_xpath('.//a[@class="nav next ui_button primary"]').click()

driver.close()


Num reviews in this page: 10
['February 23, 2021', '50', 'The ambiance is low key', 'The ambiance is low key and elegant. My husband got the 16 oz steak which was juicy and well seasoned. The lobster Mac and cheese was decadent and so rich. I also got the crab cakes which were big balls that comes in 2 with like shoestring fries on top-very filling. They came out with with bread appetizer - wasn’t all that great. Pretty penny all together. The lemonade has refills but they have options for frankly squeezed lemonade and fountain, bottled water and tap, and so on. I just don’t like the lights, they’re too dim that I was practically squinting with my contacts. Everyone greeted us coming in and they have self-park and valet.']
continue? [y/n]: y
['January 8, 2021', '50', 'The Best Steakhouse', 'This is #1 in DFW area in our opinion. And the prices equal the quality - very high. But it was worth every penny to have the most amazing quality steak and lamb chops. And it is kind of cool to pic

continue? [y/n]: y
['February 18, 2020', '40', 'Good not great', "Was here last in 2015 (see my review) and had a poor visit. Its been 4 years so gave them another try. This time much better but they can do SO MUCH more! Service was slow (drinks took 20+ min) and very inconsistent. For a $100 steak I'd like someone that appears to have been working a nice restaurant more than a few months. Lobster deviled eggs were one of the best things I have ever tasted. Truly amazing. Sides were decent. Cocktails were reasonably priced. Steak was ordered medium rare and came out dripping blood and mooing. Sent it back with the stern warning that it may be overcooked. Is that really our fault/problem? Steak came back closer to Medium but we ate it. It was very good. I've been to several of the Bob's locations and its got my money from now on."]
continue? [y/n]: y
Num reviews in this page: 10
['February 12, 2020', '40', 'Old school steakhouse', 'Pappas Bros is our go-to for out of town guests. It is 

In [18]:
dir(review)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 